In [1]:
!pip install -qU accelerate peft bitsandbytes transformers trl huggingface_hub scipy tensorboard datasets evaluate mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 33.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 124.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 60.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 119.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 117.6 MB/s eta 0:00:0000:0100:01

In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    Trainer, 
    TrainingArguments,
    DataCollatorWithPadding
)
from peft import PeftModel
import evaluate
import numpy as np
from sklearn.metrics import roc_auc_score

In [4]:
# ------------------------------------------------------------------
# 1. Configuration & Paths
# ------------------------------------------------------------------
ADAPTER_MODEL_PATH = "/kaggle/input/mlflow-folder/mlflow_artifacts/student_model"
BASE_MODEL_ID = "NousResearch/Llama-3.2-1B"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [5]:
# ------------------------------------------------------------------
# 2. Load Tokenizer & Model
# ------------------------------------------------------------------
tokenizer = AutoTokenizer.from_pretrained(ADAPTER_MODEL_PATH)

base_model = AutoModelForSequenceClassification.from_pretrained(
    BASE_MODEL_ID,
    num_labels=2,
    torch_dtype=torch.float16, 
    device_map=None 
)

base_model.config.pad_token_id = tokenizer.pad_token_id
base_model.config.use_cache = False

model = PeftModel.from_pretrained(base_model, ADAPTER_MODEL_PATH)
model = model.merge_and_unload()
model.to(device)
model.eval()


config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at NousResearch/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=

In [6]:
# ------------------------------------------------------------------
# 3. Prepare Dataset
# ------------------------------------------------------------------
raw_dataset = load_dataset("stanfordnlp/sst2")

def preprocess_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=128)

tokenized_datasets = raw_dataset.map(preprocess_function, batched=True)

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [13]:
from collections import Counter
for split in ("train", "validation", "test"):
    ds = tokenized_datasets[split]
    # get label key name (could be 'labels' or 'label')
    lab_key = "labels" if "labels" in ds.column_names else ("label" if "label" in ds.column_names else None)
    if lab_key is None:
        print(f"{split}: no label column")
        continue
    labels = ds[lab_key]
    c = Counter(labels)
    n = len(labels)
    print(f"{split}: total={n}, counts={dict(c)}, percentages={{k: v/n for k,v in c.items()}}")

train: total=67349, counts={0: 29780, 1: 37569}, percentages={k: v/n for k,v in c.items()}
validation: total=872, counts={1: 444, 0: 428}, percentages={k: v/n for k,v in c.items()}
test: total=1821, counts={-1: 1821}, percentages={k: v/n for k,v in c.items()}


In [7]:
# ------------------------------------------------------------------
# 4. Metrics
# ------------------------------------------------------------------

clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    
    # Return empty metrics if no labels exist
    if labels is None or (isinstance(labels, np.ndarray) and len(labels) == 0):
        return {}
        
    core = clf_metrics.compute(predictions=preds, references=labels)
    try:
        probs = torch.nn.functional.softmax(torch.tensor(logits), dim=-1).numpy()
        pos_probs = probs[:, 1]
        auc = float(roc_auc_score(labels, pos_probs))
    except:
        auc = float("nan")
    core["roc_auc"] = auc
    return core

In [9]:
training_args = TrainingArguments(
    output_dir="./eval_results",
    per_device_eval_batch_size=32,
    report_to="none", 
)

trainer = Trainer(
    model=model,
    args=training_args,
    processing_class=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics
)

In [21]:
# ------------------------------------------------------------------
# Training Results
# ------------------------------------------------------------------

import json
import pandas as pd

# Load your JSON file
with open("/kaggle/input/mlflow-folder/distilled-llama-sst2/checkpoint-4210/trainer_state.json", "r") as f:
    data = json.load(f)

log_history = data["log_history"]

# Collect only eval steps (those having eval_accuracy, eval_loss, etc.)
rows = []
for entry in log_history:
    if "eval_accuracy" in entry:   # means this is an eval-step log
        rows.append({
            "step": entry.get("step"),
            "epoch": entry.get("epoch"),
            "eval_loss": entry.get("eval_loss"),
            "eval_accuracy": entry.get("eval_accuracy"),
            "eval_f1": entry.get("eval_f1"),
            "eval_precision": entry.get("eval_precision"),
            "eval_recall": entry.get("eval_recall"),
            "eval_roc_auc": entry.get("eval_roc_auc"),
        })

# Convert to dataframe
df = pd.DataFrame(rows)

# Show table
df


,step,epoch,eval_loss,eval_accuracy,eval_f1,eval_precision,eval_recall,eval_roc_auc
0,1000,0.237530,0.386957,0.900229,0.903654,0.888889,0.918919,0.944578
1,2000,0.475059,0.370412,0.910550,0.913525,0.899563,0.927928,0.941697
2,3000,0.712589,0.352371,0.911697,0.912797,0.917995,0.907658,0.947085
3,4000,0.950119,0.336654,0.924312,0.924658,0.937500,0.912162,0.952834


In [11]:
val_ds = tokenized_datasets["validation"]

val_ds = val_ds.rename_column("label", "labels")
val_ds.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

val_metrics = trainer.evaluate(eval_dataset=val_ds)
for key, value in val_metrics.items():
    print(f"Valid_{key}: {value:.4f}")

Valid_eval_loss: 0.3916
Valid_eval_model_preparation_time: 0.0030
Valid_eval_accuracy: 0.9220
Valid_eval_f1: 0.9224
Valid_eval_precision: 0.9352
Valid_eval_recall: 0.9099
Valid_eval_roc_auc: 0.9530
Valid_eval_runtime: 31.0736
Valid_eval_samples_per_second: 28.0620
Valid_eval_steps_per_second: 0.9010


In [15]:
import numpy as np
import torch
from tqdm.auto import tqdm

def get_logits_labels(split, batch_size=64):
    all_logits, all_labels = [], []
    ds = tokenized_datasets[split]
    # Use raw tokenized inputs (they are already torch format or lists)
    for i in range(0, len(ds), batch_size):
        batch = ds[i:i+batch_size]
        input_ids = torch.tensor(batch["input_ids"]).to(device)
        attention_mask = torch.tensor(batch["attention_mask"]).to(device)
        with torch.no_grad():
            out = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = out.logits.detach().cpu().numpy()
        all_logits.append(logits)
        all_labels.append(np.array(batch["label"]))
    all_logits = np.concatenate(all_logits, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    return all_logits, all_labels

for split in ["validation"]:
    print(f"--- {split} ---")
    logits, labels = get_logits_labels(split, batch_size=64)
    print("logits shape:", logits.shape)
    print("logits min/max/mean per class:", logits.min(axis=0), logits.max(axis=0), logits.mean(axis=0))
    probs = np.exp(logits - np.max(logits, axis=1, keepdims=True))
    probs = probs / probs.sum(axis=1, keepdims=True)
    print("probs mean per class:", probs.mean(axis=0))
    if labels is not None and labels.dtype != object:
        for lab in np.unique(labels):
            idx = np.where(labels == lab)[0]
            print(f" label={lab} count={len(idx)} logits mean for that label:", logits[idx].mean(axis=0), "probs mean:", probs[idx].mean(axis=0))
    print()


--- validation ---
logits shape: (872, 2)
logits min/max/mean per class: [-1.876 -1.97 ] [0.83  1.158] [-0.4514 -0.639 ]
probs mean per class: [0.534  0.4656]
 label=0 count=428 logits mean for that label: [ 0.00566 -1.101  ] probs mean: [0.734 0.266]
 label=1 count=444 logits mean for that label: [-0.892  -0.1941] probs mean: [0.342  0.6577]



In [17]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report

def inspect_split(split, n_show=10):
    logits, labels = get_logits_labels(split, batch_size=64)
    probs = np.exp(logits - np.max(logits, axis=1, keepdims=True))
    probs = probs / probs.sum(axis=1, keepdims=True)
    preds = np.argmax(logits, axis=1)
    print("Confusion Matrix for", split)
    if labels is None or labels.dtype == object:
        print("No labels available for this split.")
        return preds, labels
    cm = confusion_matrix(labels, preds)
    print(cm)
    print("Classification report:\n", classification_report(labels, preds, digits=4))
    return preds, labels

_ = inspect_split("validation", n_show=8)

Confusion Matrix for validation
[[400  28]
 [ 40 404]]
Classification report:
               precision    recall  f1-score   support

           0     0.9091    0.9346    0.9217       428
           1     0.9352    0.9099    0.9224       444

    accuracy                         0.9220       872
   macro avg     0.9221    0.9222    0.9220       872
weighted avg     0.9224    0.9220    0.9220       872

